In [43]:
import argparse
import roypy
import time
import queue
from sample_camera_info import print_camera_info
from roypy_sample_utils import CameraOpener, add_camera_opener_options
#from roypy_platform_utils import PlatformHelper

import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
import math
import socket_sender

try:
    import roypycy
except ImportError:
    print("Pico Flexx backend requirements (roypycy) not installed properly")
    raise

# RANSAM

In [2]:
# # RANSAM

def Dis_pt2plane(pts, a, b, c, d):
    """
    Compute the distance from points to the plane
    """
    normal = math.sqrt(a*a+b*b+c*c)
    if normal == 0:
        normal = 1
    
    v = np.array([a,b,c])
    dis = abs(np.dot(pts,v.T)+d)/normal
    return dis

def get_Plane(sampts):
    """
    Compute the equation of the plane
    """
    p1 = sampts[0]
    p2 = sampts[1]
    p3 = sampts[2]
    
    a = ( (p2[1]-p1[1])*(p3[2]-p1[2])-(p2[2]-p1[2])*(p3[1]-p1[1]) )
    b = ( (p2[2]-p1[2])*(p3[0]-p1[0])-(p2[0]-p1[0])*(p3[2]-p1[2]) )
    c = ( (p2[0]-p1[0])*(p3[1]-p1[1])-(p2[1]-p1[1])*(p3[0]-p1[0]) )
    d = ( 0-(a*p1[0]+b*p1[1]+c*p1[2]) )
    
    return a,b,c,d

# def Random3points(points3D, ConfidenceIndex):
#     """
#     Random choose 3 Confidence points
#     """
#     sample_number = 3
#     sample_point_index = random.sample(range(ConfidenceIndex.shape[0]), sample_number)
#     sample_points = np.zeros((sample_number,3))
#     for i in range(sample_number):
#         Confidence_point_index = sample_point_index[i]
#         index = ConfidenceIndex[Confidence_point_index]
#         y = index // points3D.shape[1]
#         x = index % points3D.shape[1]
#         sample_points[i] = points3D[y][x]
#     return sample_points

def Random3points(points3D):
    sample_number = 20
    sample_point_index = random.sample(range(points3D.shape[0]*points3D.shape[1]), sample_number)
    sample_points = np.zeros((3,3))
    num = 0
    for i in range(sample_number):
        index = sample_point_index[i]
        y = index // points3D.shape[1]
        x = index % points3D.shape[1]
        
        point = points3D[y][x]
        if(point[0] != 0 or point[1] != 0 or point[2] != 0):
            sample_points[num] = points3D[y][x]
            num = num + 1
        
        if(num == 3):
            break
    return sample_points

def get_inliner_num(points3D,a,b,c,d,inliner_threshold):
    """
    Compute the liner points which distance to plane < threshold
    Also get distance from points to the plane (new Depth Image which re-project depth pixels in surface plane)
    """
    inliner_num = 0
    
    dist = Dis_pt2plane(points3D,a,b,c,d)
    inliner_mask = dist < inliner_threshold
    inliner_num = np.sum(inliner_mask)
    return inliner_num, inliner_mask, dist

def RANSAM(points3D, ransac_iteration = 1000, inliner_threshold = 0.01):
    best_inlinernum = -1
    best_inlinernum = 0
    best_plane = np.zeros((1,4))
    best_depthImage = np.zeros((points3D.shape[0],points3D.shape[1]))
    best_plane_mask = np.zeros((points3D.shape[0],points3D.shape[1]))
    best_sampts = np.zeros((3,3))
    
#     print(points3D.shape,points3D[80:90,110])
    for i in range(ransac_iteration):
        sampts = Random3points(points3D)
        a,b,c,d = get_Plane(sampts)
        
        inliner_num, inliner_mask, depthImage = get_inliner_num(points3D,a,b,c,d,inliner_threshold)
        if(inliner_num > best_inlinernum):
            best_inlinernum = inliner_num
            best_plane = np.array([a,b,c,d])
            best_plane_mask = inliner_mask
            best_depthImage = depthImage
            best_sampts = sampts
            
    print("Inliner Number\n", best_inlinernum)
    print("Inliner plane\n", best_plane)
    return best_plane, best_depthImage, best_plane_mask, best_sampts

# Draw 3D plane

In [31]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def show_plane(plane_eq):
    a,b,c,d = plane_eq[0], plane_eq[1], plane_eq[2], plane_eq[3]
    x = np.linspace(-1,1,10)
    y = np.linspace(-1,1,10)

    X,Y = np.meshgrid(x,y)
    Z = (d - a*X - b*Y) / c

    fig = plt.figure()
    ax = fig.gca(projection='3d')

    surf = ax.plot_surface(X, Y, Z)

# Tranform Data

In [ ]:
def send_plane_eq(plane_eq):
    data = ""
    temp = ['1 ']
    
    temp.append("%d %d %d %d"%(plane_eq[0], plane_eq[1], plane_eq[2], plane_eq[3]))
    data = data.join(temp)
    print('send plane_eq:', data)
    socket_sender.send(data)
    
def send_sample_points(sample_points):
    data = ""
    temp = ['1 ']
    for i in range(3): 
        temp.append("%d %d %d "%(sample_points[i][0], sample_points[i][1], sample_points[i][2]))
    data = data.join(temp)
    print('send sample point:', data)
    socket_sender.send(data)

# Main

In [45]:


class MyListener(roypy.IDepthDataListener):
    def __init__(self, xqueue, yqueue, zqueue):
        super(MyListener, self).__init__()
        self.xqueue = xqueue
        self.yqueue = yqueue
        self.zqueue = zqueue
        self.Listening = True

    def onNewData(self, data):   
        if(self.Listening):
            t_time = time.time()
            
            xvalues = []
            yvalues = []
            zvalues = []
            
            values = roypycy.get_backend_data(data)

            xvalues = values.x
            yvalues = values.y
            zvalues = values.z

            xarray = np.asarray(xvalues)
            yarray = np.asarray(yvalues)
            zarray = np.asarray(zvalues)
            
            
            q_x = xarray.reshape (-1, data.width)        
            self.xqueue.put(q_x)
            q_y = yarray.reshape (-1, data.width)        
            self.yqueue.put(q_y)
            q_z = zarray.reshape (-1, data.width)        
            self.zqueue.put(q_z)
            print('store time:', (time.time()-t_time))

    def paint (self, data, name):
        """Called in the main thread, with data containing one of the items that was added to the
        queue in onNewData.
        """
        cv2.namedWindow(name, cv2.WINDOW_NORMAL)
        cv2.imshow(name, data)
        cv2.waitKey(1)


def main ():
    parser = argparse.ArgumentParser (usage = __doc__)
    add_camera_opener_options (parser)
    parser.add_argument ("--seconds", type=int, default=15, help="duration to capture data")
    
    Replay = False
    if(Replay == True):
        options = parser.parse_args(args=['--rrf', '0108.rrf','--seconds', '5'])
    else:
        options = parser.parse_args(args=['--seconds', '10'])

    opener = CameraOpener (options)
    cam = opener.open_camera ()
    
    if(Replay == False):
        cam.setUseCase('MODE_5_35FPS_600')#MODE_9_5FPS_2000 MODE_5_45FPS_500

    #Print camera information
    print_camera_info (cam)
    print("isConnected", cam.isConnected())
    print("getFrameRate", cam.getFrameRate())
    print("UseCase",cam.getCurrentUseCase())

    # we will use this queue to synchronize the callback with the main
    # thread, as drawing should happen in the main thread 
    x = queue.LifoQueue()
    y = queue.LifoQueue()
    z = queue.LifoQueue()
    l = MyListener(x,y,z)
    cam.registerDataListener(l)
    cam.startCapture()
    
    # create a loop that will run for a time (default 15 seconds)
    process_event_queue (x, y, z, l, options.seconds)
    cam.stopCapture()
    
    cv2.destroyAllWindows()
    
#改成按按鍵結束
def process_event_queue (x,y,z, painter, seconds):
    # create a loop that will run for the given amount of time
    t_end = time.time() + seconds
    while time.time() < t_end:
        try:
            # try to retrieve an item from the queue.
            # this will block until an item can be retrieved
            # or the timeout of 1 second is hit
            t_time = time.time()
            
            item_x = x.get(True, 0.5)
            item_y = y.get(True, 0.5)
            item_z = z.get(True, 0.5)
            points3D = np.dstack((item_x,item_y,item_z))
            print('queue time:', (time.time()-t_time))
        except queue.Empty:
            # this will be thrown when the timeout is hit
            break
        else:
            painter.paint (item_z, 'Depth')
            t_time = time.time()
#             surface_plane, depthImg, plane_mask, best_sampts = RANSAM(points3D, ConfidenceIndex, ransac_iteration = 500, inliner_threshold = 0.003)
            surface_plane, depthImg, plane_mask, best_sampts = RANSAM(points3D, ransac_iteration = 50, inliner_threshold = 0.003)
            print('Ransam time:', (time.time()-t_time))
            painter.paint (plane_mask.astype(np.uint8)*255, 'plane')
            
            #Send surface_plane and best_sampts
            send_plane_eq(surface_plane)
            send_sample_points(best_sampts)
            
            show_3d_plane_img = False  
            if(show_3d_plane_img):
                show_plane(surface_plane)
                


In [46]:


main()


Number of cameras connected:  1
        Camera information
Type:            PICOFLEXX
Width:           224
Height:          171
Operation modes: 10
    MODE_9_5FPS_2000
    MODE_9_10FPS_1000
    MODE_9_15FPS_700
    MODE_9_25FPS_450
    MODE_5_35FPS_600
    MODE_5_45FPS_500
    MODE_MIXED_30_5
        this operation mode has 2 streams
    MODE_MIXED_50_5
        this operation mode has 2 streams
    Low_Noise_Extended
    Fast_Acquisition
Lens parameters: 9
    ('cx', 118.28559112548828)
    ('cy', 87.74105072021484)
    ('fx', 213.8031768798828)
    ('fy', 213.8031768798828)
    ('k1', 0.4155448377132416)
    ('k2', -4.7316107749938965)
    ('k3', 8.45906925201416)
    ('p1', 7.605663946304829e-16)
    ('p2', 4.939198934392371e-16)
CameraInfo items: 8
    ('BRIDGE_TYPE', 'Enclustra')
    ('MODULE_IDENTIFIER', '00000000')
    ('MODULE_IDENTIFIER_HASH', '558161692')
    ('MODULE_SERIAL', '0')
    ('MODULE_SUFFIX', '')
    ('IMAGER', 'M2450_A12_AIO')
    ('PROCESSING_NAME', 'Spectre')
  

store time: 0.0009889602661132812
queue time: 0.0
store time: 0.0009663105010986328
store time: 0.0009932518005371094
Inliner Number
 4913
Inliner plane
 [ 0.00366238 -0.0055554  -0.01357574  0.00529542]
Ransam time: 0.07184028625488281
queue time: 0.0
store time: 0.0
store time: 0.0009717941284179688
store time: 0.0009658336639404297
Inliner Number
 4652
Inliner plane
 [-0.00276198  0.00623111  0.01881178 -0.00722364]
Ransam time: 0.07371091842651367
queue time: 0.0
store time: 0.0009980201721191406
store time: 0.0009903907775878906
store time: 0.0009911060333251953
Inliner Number
 4031
Inliner plane
 [-0.00457109  0.00646492  0.01895659 -0.00748391]
Ransam time: 0.07570600509643555
queue time: 0.0
store time: 0.000997304916381836
store time: 0.0
Inliner Number
 3879
Inliner plane
 [-0.0006926   0.00143694  0.0054742  -0.00208461]
Ransam time: 0.0688166618347168
queue time: 0.0
store time: 0.0
store time: 0.000997781753540039
store time: 0.0009975433349609375
Inliner Number
 5010
Inli

store time: 0.0010042190551757812
store time: 0.0009667873382568359
Inliner Number
 8235
Inliner plane
 [ 2.50826297e-03 -5.51849636e-03 -1.78685975e-04 -1.46527095e-05]
Ransam time: 0.07374429702758789
queue time: 0.0
store time: 0.0010297298431396484
store time: 0.0009646415710449219
store time: 0.0009970664978027344
Inliner Number
 5124
Inliner plane
 [-0.00357773  0.0031306   0.00884638 -0.00349346]
Ransam time: 0.07283282279968262
queue time: 0.0
store time: 0.0
store time: 0.0009980201721191406
store time: 0.001994609832763672
Inliner Number
 4814
Inliner plane
 [-0.00403663  0.00349423  0.00967486 -0.00384224]
Ransam time: 0.07777953147888184
queue time: 0.0
store time: 0.000997304916381836
store time: 0.0009970664978027344
store time: 0.0009965896606445312
Inliner Number
 5250
Inliner plane
 [ 0.00012646 -0.00014425 -0.00034218  0.00013684]
Ransam time: 0.08676743507385254
queue time: 0.0
store time: 0.0
store time: 0.0009970664978027344
store time: 0.000997781753540039
Inliner

store time: 0.0009984970092773438
store time: 0.0
store time: 0.000997781753540039
Inliner Number
 5685
Inliner plane
 [-0.00397147  0.00458659  0.01282934 -0.00498254]
Ransam time: 0.07779145240783691
queue time: 0.0
store time: 0.0
store time: 0.0
Inliner Number
 5097
Inliner plane
 [-0.00721314  0.0073687   0.02678657 -0.01016675]
Ransam time: 0.07080841064453125
queue time: 0.0
store time: 0.0009970664978027344
store time: 0.0019943714141845703
store time: 0.000997781753540039
Inliner Number
 5514
Inliner plane
 [-0.0047303   0.00383156  0.01389558 -0.00538525]
Ransam time: 0.07280564308166504
queue time: 0.0
store time: 0.000995635986328125
store time: 0.0009951591491699219
store time: 0.0009982585906982422
Inliner Number
 5881
Inliner plane
 [ 0.00352761 -0.0033166  -0.01093983  0.00427149]
Ransam time: 0.07579469680786133
queue time: 0.0
store time: 0.0009975433349609375
store time: 0.0
store time: 0.0
Inliner Number
 5504
Inliner plane
 [ 0.00517515 -0.00621148 -0.01734631  0.0

In [28]:
# !jupyter nbconvert --to script HMD_Light_main.ipynb

[NbConvertApp] Converting notebook HMD_Light_main.ipynb to script
[NbConvertApp] Writing 7796 bytes to HMD_Light_main.py
